In [52]:
%pip install -q amplpy

In [53]:
from amplpy import AMPL, tools

ampl = tools.ampl_notebook(
    modules=["highs"],  # modules to install
    license_uuid="default",  # license to use
)

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://colab.ampl.com).


In [58]:
%%writefile MaquinasParalelasMakespan.mod
set TAREFAS;
set MAQUINAS;
set MATRIZ1 = {TAREFAS,MAQUINAS};
set MATRIZ2 = {TAREFAS,TAREFAS, MAQUINAS};

param p {MATRIZ1};
param d {TAREFAS};
param G := 10000;

var Cmax >= 0;
var E {TAREFAS} >= 0;
var C {MATRIZ1} >= 0;
var X {MATRIZ2} binary;
var Etotal >= 0;

minimize FO: Cmax;

subject to R1 {j in TAREFAS: j <> 0}:
  sum {k in MAQUINAS, i in TAREFAS} X[i,j,k] = 1;

subject to R2 {k in MAQUINAS}:
  sum {j in TAREFAS: j <> 0} X[0,j,k] <= 1;

subject to R3 {h in TAREFAS, k in MAQUINAS: h <> 0}:
  sum {i in TAREFAS: i <> h} X[i,h,k] - sum {j in TAREFAS: j <> h} X[h,j,k] = 0;

subject to R4 {i in TAREFAS, j in TAREFAS, k in MAQUINAS: j <> 0}:
  C[j,k] >= C[i,k] - G + (p[j,k] + G)*X[i,j,k];

subject to R5 {i in TAREFAS, k in MAQUINAS: i <> 0}:
  Cmax >= C[i,k];

subject to R6 {i in TAREFAS, k in MAQUINAS: i <> 0}:
  E[i] >= d[i] - C[i,k];

subject to R7:
  Etotal = sum{i in TAREFAS: i <> 0} E[i];


Overwriting MaquinasParalelasMakespan.mod


In [55]:
%%writefile MaquinasParalelasMakespan.dat
data;
set TAREFAS := 0 1 2 3 4 5 6;
set MAQUINAS := 1 2 3;
param: p :=
  0 1 0
  0 2 0
  0 3 0
  1 1 6
  1 2 5
  1 3 5
  2 1 10
  2 2 11
  2 3 13
  3 1 7
  3 2 8
  3 3 9
  4 1 13
  4 2 11
  4 3 16
  5 1 23
  5 2 25
  5 3 24
  6 1 19
  6 2 14
  6 3 18;

param: d :=
  0 0
  1 27
  2 38
  3 43
  4 29
  5 19
  6 35;

Writing MaquinasParalelasMakespan.dat


In [59]:
%%ampl_eval
model MaquinasParalelasMakespan.mod;
data MaquinasParalelasMakespan.dat;
option solver highs;
solve;
display Cmax;
display X;
display C;
reset;


HiGHS 1.5.3: HiGHS 1.5.3: optimal solution; objective 25
10043 simplex iterations
1197 branching nodes
absmipgap=0.0004, relmipgap=1.6e-05
Cmax = 25

X [*,*,1]
:   0   1   2        3         4   5   6    :=
0   0   1   0   -2.82337e-14   0   0   0
1   0   0   1    0             0   0   0
2   0   0   0    1             0   0   0
3   1   0   0    0             0   0   0
4   0   0   0    0             0   0   0
5   0   0   0    0             0   0   0
6   0   0   0    0             0   0   0

 [*,*,2]
:   0   1        2              3         4   5        6          :=
0   0   0    7.02216e-15    0             0   0    1
1   0   0    0              0             0   0    7.10543e-15
2   0   0    0             -1.15463e-14   0   0   -7.10543e-15
3   0   0    0              0             0   0    0
4   1   0   -2.57572e-14    0             0   0    0
5   0   0    0              0             0   0    0
6   0   0    0              1.15463e-14   1   0    0

 [*,*,3]
:   0        